In [37]:
import nfl_data_py as nfl
import pandas as pd
import xgboost as xgb
import numpy as np
import math

In [24]:
pbp = nfl.import_pbp_data([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


In [25]:
pbp.shape

(390566, 384)

In [13]:
pbp.head

<bound method NDFrame.head of        play_id          game_id old_game_id home_team away_team season_type  \
0          1.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
1         43.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
2         68.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
3         89.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
4        115.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
...        ...              ...         ...       ...       ...         ...   
50142   4035.0   2022_22_KC_PHI  2023021200       PHI        KC        POST   
50143   4024.0   2022_22_KC_PHI  2023021200       PHI        KC        POST   
50144   4050.0   2022_22_KC_PHI  2023021200       PHI        KC        POST   
50145   4072.0   2022_22_KC_PHI  2023021200       PHI        KC        POST   
50146   4094.0   2022_22_KC_PHI  2023021200       PHI        KC        POST   

       week posteam p

In [17]:
data = pbp.columns
data

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'offense_formation', 'offense_personnel', 'defenders_in_box',
       'defense_personnel', 'number_of_pass_rushers', 'players_on_play',
       'offense_players', 'defense_players', 'n_offense', 'n_defense'],
      dtype='object', length=384)

In [18]:
for column in data:
    print(column)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


In [83]:
# Get game_id for first game in dataset
game_id = pbp['game_id'].iloc[4]

In [84]:
# get all the plays for the first game
game_plays = pbp[pbp['game_id'] == game_id]
game_plays
# save to csv
game_plays.to_csv('game_plays2.csv')

In [78]:
pre_play_params = ["posteam_type", "yardline_100", "half_seconds_remaining", "game_seconds_remaining", "down", "ydstogo", "posteam_timeouts_remaining", "defteam_timeouts_remaining", "score_differential", "weather", "roof", "spread_line", "home_opening_kickoff" ]

# Get additional columns from the data

In [79]:
pbp['pos_spread'] = np.where(pbp['posteam_type'] == 'away', pbp['spread_line'], -pbp['spread_line'])
pbp['pos_spread_time'] = pbp.apply(lambda row: row['pos_spread'] * math.exp(-4 * (3600 - row['game_seconds_remaining']) / 3600), axis=1)
pbp["diff_time_ratio"] = pbp.apply(lambda row: row["score_differential"] * math.exp(4 * (3600 - row["game_seconds_remaining"]) / 3600), axis=1)
pbp["pos_2h_ko"] = np.where(pbp["posteam_type"] == "away", pbp["home_opening_kickoff"], 1 - pbp["home_opening_kickoff"])

# Get the winner of the game

In [80]:
pbp["home_result"] = np.where(pbp["home_score"] > pbp["away_score"], 1, 0)
pbp["home_result"] = np.where(pbp["home_score"] == pbp["away_score"], 0.5, pbp["home_result"])
pbp["pos_result"] = np.where(pbp["posteam"] == pbp["home_team"], pbp["home_result"], 1 - pbp["home_result"])

In [81]:
game_plays = pbp[pbp['game_id'] == game_id]
game_plays.to_csv('game_plays3.csv')

In [ ]:
params = ["posteam_type", "yardline_100", "half_seconds_remaining", "game_seconds_remaining", "down", "ydstogo", "posteam_timeouts_remaining", "defteam_timeouts_remaining", "score_differential", "pos_spread_time", "diff_time_ratio", "pos_2h_ko"]